<a href="https://colab.research.google.com/github/jakii1/Jakov_Aleric_zadatak1/blob/main/Kopija_bilje%C5%BEnice_main_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Inicijalizacija FastAPI aplikacije, kreacija modela te get endpoint-a

In [ ]:
from fastapi import FastAPI, Query, Path
from pydantic import BaseModel
from typing import List, Literal, Optional
import json
import httpx
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok.connect
uvicorn.run


nest_asyncio.apply()


# Inicijalizacija FastAPI aplikacije
app = FastAPI()

# Definicija modela Ticket koristeći Pydantic
class Ticket(BaseModel):
  id: int
  title: str
  status: Literal['closed', 'open']
  priority: Literal['low', 'medium', 'high']
  assignee: str


# Asinkrona funkcija za dohvat podataka o "todos" sa vanjskog API-ja
async def fetch_todos() -> List[Ticket]:
  async with httpx.AsyncClient() as client:
    response = await client.get('https://dummyjson.com/todos')
    data = response.json()['todos']
    tickets = []

    for item in data:

      id = item['id']

      title = item['todo']

      if item['completed']: # Corrected typo here
        status = 'closed'
      else:
        status = 'open'

      if item['id'] % 3 == 0:
        priority = 'low'
      elif item['id'] % 2 == 0:
        priority = 'medium'
      else:
        priority = 'high'

      assignee = item['userId']

      #Dodavanje ticketa u listu
      tickets.append(Ticket(id=id, title=title, status=status, priority=priority, assignee=str(assignee)))

    return tickets

#get endpoint 1
@app.get('/tickets', response_model = List[Ticket])
async def get_tickets():
  return await fetch_todos()

#get endpoint 2
@app.get('/tickets/{id}', response_model = Ticket) # Corrected syntax here
async def get_ticket(id: int):
  tickets = await fetch_todos()
  for ticket in tickets:
    if ticket.id == id:
      return ticket

#get endpoint 3
@app.get('/filtered_tickets', response_model = List[Ticket])
async def get_filtered_tickets(
    status: Optional[Literal['closed', 'open']] = Query(None),
    priority: Optional[Literal['low', 'medium', 'high']] = Query(None)
):
  tickets = await fetch_todos()
  if status:
    tickets = [ticket for ticket in tickets if ticket.status == status]
  if priority:
    tickets = [ticket for ticket in tickets if ticket.priority == priority]
  return tickets

#get endpoint 4
@app.get('/tickets/search', response_model = List[Ticket])
async def search_tickets(q: str = Query(...)):
    tickets = await fetch_todos()
    return [ticket for ticket in tickets if q.lower() in ticket.title.lower()]

public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")
uvicorn.run(app, host="0.0.0.0", port=8000)

Testiranje endpointa

In [ ]:
import pytest
from httpx import AsyncClient

#kreiranje asinkrong HTTP klijenta
@pytest.fixture(scope="session")
async def client():
    async with AsyncClient(app=app, base_url="http://test") as client:
        yield client

#test endpointa
async def test_get_tickets(client):
    response = await client.get("/tickets")
    assert response.status_code == 200
    assert isinstance(response.json(), list)

async def test_get_ticket_by_id(client):
    tickets_response = await client.get("/tickets")
    ticket_id = tickets_response.json()[0]['id']

    response = await client.get(f"/tickets/{ticket_id}")
    assert response.status_code == 200
    assert isinstance(response.json(), dict)
    assert response.json()['id'] == ticket_id

async def test_get_filtered_tickets_by_status(client):
    response_open = await client.get("/filtered_tickets?status=open")
    assert response_open.status_code == 200
    assert isinstance(response_open.json(), list)
    for ticket in response_open.json():
        assert ticket['status'] == 'open'

    response_closed = await client.get("/filtered_tickets?status=closed")
    assert response_closed.status_code == 200
    assert isinstance(response_closed.json(), list)
    for ticket in response_closed.json():
        assert ticket['status'] == 'closed'

async def test_get_filtered_tickets_by_priority(client):
    response_low = await client.get("/filtered_tickets?priority=low")
    assert response_low.status_code == 200
    assert isinstance(response_low.json(), list)
    for ticket in response_low.json():
        assert ticket['priority'] == 'low'

    response_medium = await client.get("/filtered_tickets?priority=medium")
    assert response_medium.status_code == 200
    assert isinstance(response_medium.json(), list)
    for ticket in response_medium.json():
        assert ticket['priority'] == 'medium'

    response_high = await client.get("/filtered_tickets?priority=high")
    assert response_high.status_code == 200
    assert isinstance(response_high.json(), list)
    for ticket in response_high.json():
        assert ticket['priority'] == 'high'

async def test_search_tickets(client):
    # Get a ticket title to search for
    tickets_response = await client.get("/tickets")
    search_term = tickets_response.json()[0]['title'].split()[0] # Use the first word of a title

    response = await client.get(f"/tickets/search?q={search_term}")
    assert response.status_code == 200
    assert isinstance(response.json(), list)
    for ticket in response.json():
        assert search_term.lower() in ticket['title'].lower()